In [25]:
import pandas as pd
import requests
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import datetime

from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import MobileApplicationClient

In [26]:
# This is needed to authenticate us to
import os
%load_ext dotenv
%dotenv
try:
    from requests_oauthlib import OAuth2
except ModuleNotFoundError:
    import sys
    import os

    # I need this because requests_oauth gets installed in a weird place on my system
    sys.path.append('/usr/local/lib/python3.6/site-packages')
    from requests_oauthlib import OAuth2

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [27]:
OAuth2

requests_oauthlib.oauth2_auth.OAuth2

In [28]:
access_token = os.getenv("ACCESS_TOKEN")
client_secret = os.getenv("CLIENT_SECRET")
client_id = os.getenv("CLIENT_ID")
callback_url = os.getenv("CALLBACK_URL")
authorization_uri = os.getenv("AUTHORIZATION_URI")

In [16]:
# %load ~/anaconda3/lib/python3.6/site-packages/requests_oauthlib/oauth2_auth.py# %load 
from __future__ import unicode_literals
from oauthlib.oauth2 import WebApplicationClient, InsecureTransportError
from oauthlib.oauth2 import is_secure_transport
from requests.auth import AuthBase


class OAuth2(AuthBase):
    """Adds proof of authorization (OAuth2 token) to the request."""

    def __init__(self, client_id=None, client=None, token=None):
        """Construct a new OAuth 2 authorization object.

        :param client_id: Client id obtained during registration
        :param client: :class:`oauthlib.oauth2.Client` to be used. Default is
                       WebApplicationClient which is useful for any
                       hosted application but not mobile or desktop.
        :param token: Token dictionary, must include access_token
                      and token_type.
        """
        self._client = client or WebApplicationClient(client_id, token=token)
        if token:
            for k, v in token.items():
                setattr(self._client, k, v)

    def __call__(self, r):
        """Append an OAuth 2 token to the request.

        Note that currently HTTPS is required for all requests. There may be
        a token type that allows for plain HTTP in the future and then this
        should be updated to allow plain HTTP on a white list basis.
        """
        if not is_secure_transport(r.url):
            raise InsecureTransportError()
        r.url, r.headers, r.body = self._client.add_token(r.url,
                http_method=r.method, body=r.body, headers=r.headers)
        return r

In [32]:
scope = ["activity", "heartrate", "location", "nutrition", "profile", "settings", "sleep", "social", "weight"]

client = MobileApplicationClient(client_id)

token = {
    'access_token': access_token,
    'token_type': 'Bearer'
}

fitbit = OAuth2Session(client_id, client=client, scope=scope, token=token)

In [33]:
r = fitbit.get('https://api.fitbit.com/1/user/-/profile.json')

In [35]:
response = r.json()

In [36]:
response

{'user': {'age': 25,
  'ambassador': False,
  'autoStrideEnabled': True,
  'avatar': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_100_square.jpg',
  'avatar150': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_150_square.jpg',
  'avatar640': 'https://d6y8zfzc2qfsl.cloudfront.net/9A7CB7C7-776B-56AF-D8CF-6A0AC63C1A2B_profile_640_square.jpg',
  'averageDailySteps': 7822,
  'clockTimeDisplayFormat': '12hour',
  'corporate': False,
  'corporateAdmin': False,
  'country': 'US',
  'dateOfBirth': '1992-12-11',
  'displayName': 'Brenden R.',
  'displayNameSetting': 'name',
  'distanceUnit': 'en_US',
  'encodedId': '2F5M36',
  'familyGuidanceEnabled': False,
  'features': {'exerciseGoal': True},
  'firstName': 'Brenden',
  'foodsLocale': 'en_US',
  'fullName': 'Brenden Rossin',
  'gender': 'MALE',
  'glucoseUnit': 'en_US',
  'height': 180.4,
  'heightUnit': 'en_US',
  'isChild': False,
  'isGraduationAvailable': False,
  'l

In [101]:
yesterday = datetime.datetime(2018,8,27)

date_list = ['2018-08-27']

for i in range(19):
    date = yesterday - datetime.timedelta(days = 31)
    date_list.append(date.strftime('%Y-%m-%d'))
    yesterday = date

In [102]:
date_list

['2018-08-27',
 '2018-07-27',
 '2018-06-26',
 '2018-05-26',
 '2018-04-25',
 '2018-03-25',
 '2018-02-22',
 '2018-01-22',
 '2017-12-22',
 '2017-11-21',
 '2017-10-21',
 '2017-09-20',
 '2017-08-20',
 '2017-07-20',
 '2017-06-19',
 '2017-05-19',
 '2017-04-18',
 '2017-03-18',
 '2017-02-15',
 '2017-01-15']

In [117]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/steps/date/{value}/1m.json')
    response = r.json()
    steps = response['activities-tracker-steps']
    steps_df = pd.DataFrame(steps)
    if index == 0:
        all_steps_df = steps_df
    else:
        all_steps_df = pd.concat([all_steps_df, steps_df])

all_steps_df.to_pickle("Data/fitbit_steps.pkl")

all_steps_df.rename(index=str, columns={"dateTime": "Date", "value": "Steps"}, inplace=True)

all_steps_df

,Date,Steps
0,2018-07-28,4031
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,3985
4,2018-08-01,4173
5,2018-08-02,7469
6,2018-08-03,10066
7,2018-08-04,8927
8,2018-08-05,2260
9,2018-08-06,7036


In [118]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/distance/date/{value}/1m.json')
    response = r.json()
    distance = response['activities-tracker-distance']
    distance_df = pd.DataFrame(distance)
    if index == 0:
        all_distance_df = distance_df
    else:
        all_distance_df = pd.concat([all_distance_df, distance_df])
    

all_distance_df.to_pickle("Data/fitbit_distance.pkl")    

all_distance_df.rename(index=str, columns={"dateTime": "Date", "value": "Distance"}, inplace=True)

all_distance_df

,Date,Distance
0,2018-07-28,3.0192099999999997
1,2018-07-29,0.0
2,2018-07-30,0.0
3,2018-07-31,2.98476
4,2018-08-01,3.1255699999999997
5,2018-08-02,5.5942799999999995
6,2018-08-03,7.539429999999999
7,2018-08-04,7.362139999999999
8,2018-08-05,1.69274
9,2018-08-06,5.26996


In [119]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/calories/date/{value}/1m.json')
    response = r.json()
    calories = response['activities-tracker-calories']
    calories_df = pd.DataFrame(calories)
    if index == 0:
        all_calories_df = calories_df
    else:
        all_calories_df = pd.concat([all_calories_df, calories_df])
    

all_calories_df.to_pickle("Data/fitbit_calories.pkl")    

all_calories_df.rename(index=str, columns={"dateTime": "Date", "value": "Calories_Burned"}, inplace=True)

all_calories_df

,Date,Calories_Burned
0,2018-07-28,2237
1,2018-07-29,1851
2,2018-07-30,1851
3,2018-07-31,2486
4,2018-08-01,2434
5,2018-08-02,3030
6,2018-08-03,3030
7,2018-08-04,2838
8,2018-08-05,2193
9,2018-08-06,2723


In [121]:
fitbit_df = all_steps_df.merge(all_distance_df, left_on='Date', right_on='Date', how='outer')
fitbit_df = fitbit_df.merge(all_calories_df, left_on='Date', right_on='Date', how='outer')

fitbit_df

,Date,Steps,Distance,Calories_Burned
0,2018-07-28,4031,3.0192099999999997,2237
1,2018-07-29,0,0.0,1851
2,2018-07-30,0,0.0,1851
3,2018-07-31,3985,2.98476,2486
4,2018-08-01,4173,3.1255699999999997,2434
5,2018-08-02,7469,5.5942799999999995,3030
6,2018-08-03,10066,7.539429999999999,3030
7,2018-08-04,8927,7.362139999999999,2838
8,2018-08-05,2260,1.69274,2193
9,2018-08-06,7036,5.26996,2723


In [132]:
fitbit_df.Date = pd.to_datetime(fitbit_df.Date)
fitbit_df.Steps = fitbit_df.Steps.astype('int64')
fitbit_df.Distance = fitbit_df.Distance.astype('float')
fitbit_df.Calories_Burned = fitbit_df.Calories_Burned.astype('int64')

In [133]:
fitbit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620 entries, 0 to 619
Data columns (total 4 columns):
Date               620 non-null datetime64[ns]
Steps              620 non-null int64
Distance           620 non-null float64
Calories_Burned    620 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 24.2 KB


In [134]:
fitbit_df.head()

,Date,Steps,Distance,Calories_Burned
0,2018-07-28,4031,3.01921,2237
1,2018-07-29,0,0.00000,1851
2,2018-07-30,0,0.00000,1851
3,2018-07-31,3985,2.98476,2486
4,2018-08-01,4173,3.12557,2434


In [136]:
len(fitbit_df[fitbit_df['Steps'] < 1])

51

In [137]:
fitbit_df.to_pickle("Data/fitbit_stepsdistancecals.pkl")    

In [138]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/floors/date/{value}/1m.json')
    response = r.json()
    floors = response['activities-tracker-floors']
    floors_df = pd.DataFrame(floors)
    if index == 0:
        all_floors_df = floors_df
    else:
        all_floors_df = pd.concat([all_floors_df, floors_df])
        
all_floors_df.rename(index=str, columns={"dateTime": "Date", "value": "Floors"}, inplace=True)    

all_floors_df.to_pickle("Data/fitbit_floors.pkl")    

all_floors_df

,Date,Floors
0,2018-07-28,2
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,4
4,2018-08-01,9
5,2018-08-02,10
6,2018-08-03,15
7,2018-08-04,20
8,2018-08-05,7
9,2018-08-06,7


In [139]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/elevation/date/{value}/1m.json')
    response = r.json()
    elevation = response['activities-tracker-elevation']
    elevation_df = pd.DataFrame(elevation)
    if index == 0:
        all_elevation_df = elevation_df
    else:
        all_elevation_df = pd.concat([all_elevation_df, elevation_df])
    

all_elevation_df.rename(index=str, columns={"dateTime": "Date", "value": "Elevation"}, inplace=True)

all_elevation_df.to_pickle("Data/fitbit_elevation.pkl")    

all_elevation_df

,Date,Elevation
0,2018-07-28,6
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,12
4,2018-08-01,27
5,2018-08-02,30
6,2018-08-03,45
7,2018-08-04,60
8,2018-08-05,21
9,2018-08-06,21


In [140]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/activityCalories/date/{value}/1m.json')
    response = r.json()
    activityCalories = response['activities-tracker-activityCalories']
    activityCalories_df = pd.DataFrame(activityCalories)
    if index == 0:
        all_activityCalories_df = activityCalories_df
    else:
        all_activityCalories_df = pd.concat([all_activityCalories_df, activityCalories_df])
    
all_activityCalories_df.rename(index=str, columns={"dateTime": "Date", "value": "Activity_Calories"}, inplace=True)
    
all_activityCalories_df.to_pickle("Data/fitbit_activityCalories.pkl")    

all_activityCalories_df

,Date,Activity_Calories
0,2018-07-28,0
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,0
4,2018-08-01,0
5,2018-08-02,0
6,2018-08-03,0
7,2018-08-04,0
8,2018-08-05,0
9,2018-08-06,965


In [141]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/minutesSedentary/date/{value}/1m.json')
    response = r.json()
    minutesSedentary = response['activities-tracker-minutesSedentary']
    minutesSedentary_df = pd.DataFrame(minutesSedentary)
    if index == 0:
        all_minutesSedentary_df = minutesSedentary_df
    else:
        all_minutesSedentary_df = pd.concat([all_minutesSedentary_df, minutesSedentary_df])
    

all_minutesSedentary_df.rename(index=str, columns={"dateTime": "Date", "value": "Minutes_Sedentary"}, inplace=True)
    
all_minutesSedentary_df.to_pickle("Data/fitbit_minutesSedentary.pkl")    

all_minutesSedentary_df

,Date,Minutes_Sedentary
0,2018-07-28,1440
1,2018-07-29,1440
2,2018-07-30,1440
3,2018-07-31,1440
4,2018-08-01,1440
5,2018-08-02,1440
6,2018-08-03,1440
7,2018-08-04,1440
8,2018-08-05,1440
9,2018-08-06,1251


In [142]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/minutesLightlyActive/date/{value}/1m.json')
    response = r.json()
    minutesLightlyActive = response['activities-tracker-minutesLightlyActive']
    minutesLightlyActive_df = pd.DataFrame(minutesLightlyActive)
    if index == 0:
        all_minutesLightlyActive_df = minutesLightlyActive_df
    else:
        all_minutesLightlyActive_df = pd.concat([all_minutesLightlyActive_df, minutesLightlyActive_df])
    

all_minutesLightlyActive_df.rename(index=str, columns={"dateTime": "Date", "value": "Minutes_Lightly_Active"}, inplace=True)
    
all_minutesLightlyActive_df.to_pickle("Data/fitbit_minutesLightlyActive.pkl")    

all_minutesLightlyActive_df

,Date,Minutes_Lightly_Active
0,2018-07-28,0
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,0
4,2018-08-01,0
5,2018-08-02,0
6,2018-08-03,0
7,2018-08-04,0
8,2018-08-05,0
9,2018-08-06,150


In [143]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/minutesFairlyActive/date/{value}/1m.json')
    response = r.json()
    minutesFairlyActive = response['activities-tracker-minutesFairlyActive']
    minutesFairlyActive_df = pd.DataFrame(minutesFairlyActive)
    if index == 0:
        all_minutesFairlyActive_df = minutesFairlyActive_df
    else:
        all_minutesFairlyActive_df = pd.concat([all_minutesFairlyActive_df, minutesFairlyActive_df])
    

all_minutesFairlyActive_df.rename(index=str, columns={"dateTime": "Date", "value": "Minutes_Fairly_Active"}, inplace=True)
    
all_minutesFairlyActive_df.to_pickle("Data/fitbit_minutesFairlyActive.pkl")    

all_minutesFairlyActive_df

,Date,Minutes_Fairly_Active
0,2018-07-28,0
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,0
4,2018-08-01,0
5,2018-08-02,0
6,2018-08-03,0
7,2018-08-04,0
8,2018-08-05,0
9,2018-08-06,31


In [144]:
for index, value in enumerate(date_list):
    r = fitbit.get(f'https://api.fitbit.com/1/user/-/activities/tracker/minutesVeryActive/date/{value}/1m.json')
    response = r.json()
    minutesVeryActive = response['activities-tracker-minutesVeryActive']
    minutesVeryActive_df = pd.DataFrame(minutesVeryActive)
    if index == 0:
        all_minutesVeryActive_df = minutesVeryActive_df
    else:
        all_minutesVeryActive_df = pd.concat([all_minutesVeryActive_df, minutesVeryActive_df])
    

all_minutesVeryActive_df.rename(index=str, columns={"dateTime": "Date", "value": "Minutes_Very_Active"}, inplace=True)
    
all_minutesVeryActive_df.to_pickle("Data/fitbit_minutesVeryActive.pkl")    

all_minutesVeryActive_df

,Date,Minutes_Very_Active
0,2018-07-28,0
1,2018-07-29,0
2,2018-07-30,0
3,2018-07-31,0
4,2018-08-01,0
5,2018-08-02,0
6,2018-08-03,0
7,2018-08-04,0
8,2018-08-05,0
9,2018-08-06,8


In [148]:
fitbit_df2 = all_steps_df.merge(all_distance_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_calories_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_floors_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_elevation_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_activityCalories_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_minutesSedentary_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_minutesLightlyActive_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_minutesFairlyActive_df, left_on='Date', right_on='Date', how='outer')
fitbit_df2 = fitbit_df2.merge(all_minutesVeryActive_df, left_on='Date', right_on='Date', how='outer')

fitbit_df2.head()

,Date,Steps,Distance,Calories_Burned,Floors,Elevation,Activity_Calories,Minutes_Sedentary,Minutes_Lightly_Active,Minutes_Fairly_Active,Minutes_Very_Active
0,2018-07-28,4031,3.0192099999999997,2237,2,6,0,1440,0,0,0
1,2018-07-29,0,0.0,1851,0,0,0,1440,0,0,0
2,2018-07-30,0,0.0,1851,0,0,0,1440,0,0,0
3,2018-07-31,3985,2.98476,2486,4,12,0,1440,0,0,0
4,2018-08-01,4173,3.1255699999999997,2434,9,27,0,1440,0,0,0


In [149]:
fitbit_df2.Date = pd.to_datetime(fitbit_df2.Date)
fitbit_df2.Steps = fitbit_df2.Steps.astype('int64')
fitbit_df2.Distance = fitbit_df2.Distance.astype('float')
fitbit_df2.Calories_Burned = fitbit_df2.Calories_Burned.astype('int64')
fitbit_df2.Floors = fitbit_df2.Floors.astype('int64')
fitbit_df2.Elevation = fitbit_df2.Elevation.astype('int64')
fitbit_df2.Activity_Calories = fitbit_df2.Activity_Calories.astype('int64')
fitbit_df2.Minutes_Sedentary = fitbit_df2.Minutes_Sedentary.astype('int64')
fitbit_df2.Minutes_Lightly_Active = fitbit_df2.Minutes_Lightly_Active.astype('int64')
fitbit_df2.Minutes_Fairly_Active = fitbit_df2.Minutes_Fairly_Active.astype('int64')
fitbit_df2.Minutes_Very_Active = fitbit_df2.Minutes_Very_Active.astype('int64')

In [150]:
fitbit_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620 entries, 0 to 619
Data columns (total 11 columns):
Date                      620 non-null datetime64[ns]
Steps                     620 non-null int64
Distance                  620 non-null float64
Calories_Burned           620 non-null int64
Floors                    620 non-null int64
Elevation                 620 non-null int64
Activity_Calories         620 non-null int64
Minutes_Sedentary         620 non-null int64
Minutes_Lightly_Active    620 non-null int64
Minutes_Fairly_Active     620 non-null int64
Minutes_Very_Active       620 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(9)
memory usage: 58.1 KB


In [151]:
fitbit_df2.head()

,Date,Steps,Distance,Calories_Burned,Floors,Elevation,Activity_Calories,Minutes_Sedentary,Minutes_Lightly_Active,Minutes_Fairly_Active,Minutes_Very_Active
0,2018-07-28,4031,3.01921,2237,2,6,0,1440,0,0,0
1,2018-07-29,0,0.00000,1851,0,0,0,1440,0,0,0
2,2018-07-30,0,0.00000,1851,0,0,0,1440,0,0,0
3,2018-07-31,3985,2.98476,2486,4,12,0,1440,0,0,0
4,2018-08-01,4173,3.12557,2434,9,27,0,1440,0,0,0


In [153]:
# fitbit_df2.set_index('Date', inplace=True)

fitbit_df2.head()

,Steps,Distance,Calories_Burned,Floors,Elevation,Activity_Calories,Minutes_Sedentary,Minutes_Lightly_Active,Minutes_Fairly_Active,Minutes_Very_Active
Date,,,,,,,,,,
2018-07-28,4031,3.01921,2237,2,6,0,1440,0,0,0
2018-07-29,0,0.00000,1851,0,0,0,1440,0,0,0
2018-07-30,0,0.00000,1851,0,0,0,1440,0,0,0
2018-07-31,3985,2.98476,2486,4,12,0,1440,0,0,0
2018-08-01,4173,3.12557,2434,9,27,0,1440,0,0,0


In [154]:
fitbit_df2 = fitbit_df2.sort_index()

fitbit_df2.tail()

,Steps,Distance,Calories_Burned,Floors,Elevation,Activity_Calories,Minutes_Sedentary,Minutes_Lightly_Active,Minutes_Fairly_Active,Minutes_Very_Active
Date,,,,,,,,,,
2018-08-23,10359,7.75889,3141,11,33,1454,710,159,38,52
2018-08-24,8696,6.51330,3216,17,51,1522,820,221,39,24
2018-08-25,165,0.12358,1977,0,0,109,757,15,0,0
2018-08-26,13286,11.05997,3080,25,76,1401,1247,103,9,81
2018-08-27,6360,4.76364,2801,8,24,1028,1254,134,29,23


In [155]:
fitbit_df2.to_pickle("Data/fitbit_all.pkl")    